# Hard Process

The starting point for almost any high energy particle physics MC event generator is the hard process, where the signal of inteterest to the user is generated. For example, this could be $g g \to H$, $q \bar{q} \to Z$, or $g g \to t\bar{t}$. Whatever is chosen becomes the start point for the entire event. In Pythia, the event generation roughly proceeds through the following steps.

1. Hard process is generated.
2. Resonances prouced in the hard process, like a Higgs or $Z$ are decayed.
3. Perturbative evolution of the hard process via parton showers, initial and final state, is performed.
4. Additional scatterings from the beam, multi-parton interactions (MPI) are interleaved into the perturbative evolution.
5. Partons from the shower are combined together into hadrons through the phenomenoligcal process of hadronization.
6. These hadrons are decayed.
7. Other non-perturbative effects are included, like hadronic rescattering or Bose-Einstein correlations. Sometimes these processes are interleaved with the decays.

Condensing this down even further, we have three major steps: hard process, evolution, and hadronization as shown below.

![Event factorization.](https://github.com/mcgen-ct/tutorials/blob/main/.full/mc/figures/event.png?raw=1)

In this tutorial, we work through how the hard process can be calculated numerically from scratch.

## Requirements

In this notebook, we would like to make some comparisons to Pythia to check whether we are getting reasonable answers. To do this, we need to set up our environment. First, we install and import the `wurlitzer` module. This allows programs that have C-like backends to write their output to the Python console. In short, this allows the output of Pythia to be displayed in this notebook.

In [ ]:
# Redirect the C output of Pythia to the notebook.
!pip install wurlitzer
from wurlitzer import sys_pipes_forever

sys_pipes_forever()

Next, we need to install the Pythia module.

In [ ]:
# Install and import the Pythia module.
!pip install pythia8mc
import pythia8mc as pythia8

We also have a few local utilities that we need to represent vectors and particle data.

In [ ]:
# Download the `vector` and `particle` modules.
!wget -q -N https://gitlab.com/mcgen-ct/tutorials/-/raw/main/.full/mc/vector.py
!wget -q -N https://gitlab.com/mcgen-ct/tutorials/-/raw/main/.full/mc/particle.py

# Import the necessary classes.
from vector import FourVector, Matrix
from particle import ParticleDatabase, ParticleData

We will also need some particle data. We will use the Pythia particle data, which can be read by the `pdb` class we just imported.

In [ ]:
# Download the Pythia particle database.
!wget -q -N https://gitlab.com/mcgen-ct/tutorials/-/raw/main/.full/mc/data/ParticleData.xml

# Create a particle database we can use throughout this notebook.
pdb = ParticleDatabase()

Finally, we need a random number generator. We could use one of the RNGs implemented in [`rng.ipynb`](rng.ipynb), but instead we will use the default `numpy` RNG. We also need the `math` module.

In [ ]:
# Import the `numpy` and `math` modules.
import numpy as np
import math

# Create an RNG, with a seed of 10.
rng = np.random.default_rng(10)

## Introduction

In particle physics we oftentimes collide two particles together, and then want to calculate the probability of some given final state. For example, we collide an electron and a positron and want to calculate the probability that we produce a muon and and an anti-muon in the final state, $e^- e^+ \to \mu^- \mu^+$. With quantum mechanics we can calculate the probability of transitioning from state $A$ at time $-\infty$ to state $B$ at time $+\infty$. In other words, the initial state $A$ is non-interacting in the far past and the final state $B$ is non-interacting in the far future. This probability of transitioning from $A$ to $B$ is,

$$
|_{+\infty} \langle B | A \rangle_{-\infty} |^2 = |_{+\infty} \langle b_1 \ldots b_m | a_1 \ldots a_n \rangle_{-\infty} |^2
$$

using Dirac notation. In the example $e^- e^+ \to \mu^- \mu^+$, $|A\rangle$ is the two particle state $|e^- e^+\rangle$ and $\langle B |$ is the two particle state $\langle \mu^- \mu^+|$. To calculate this probability, either $| A \rangle$ needs to be evolved to time $+\infty$ or $\langle B |$ needs to be evolved to time $-\infty$. This is done with the scattering-matrix $S$,

$$
| \Psi \rangle_{+\infty} = S^\dagger | \Psi\rangle_{-\infty}
$$

which takes a state $\Psi$ in the far future to the far past, and $S^\dagger S = 1$. The scattering-matrix can be factorised into a non-interacting component and an interacting component,

$$
|_{-\infty} \langle B | S | A \rangle_{-\infty} | = {}_{-\infty} \langle B | A \rangle_{-\infty} + \mathcal{M}_{A \to B}\mathcal{P}
$$

where the non-interacting term is zero if the initial and final state are different, including momenta, and one if identical. The interacting term is written as a matrix element, $\mathcal{M}$, and a phase-space component, $\mathcal{P}$. This matrix element encodes the particle interactions for the process.

The matrix element for a process can be calculated using functional integration, as realized by Richard Feynman. There is a monumental amount of theory behind all this that we are not discussing here, but the end result is quite beautiful and can be visually represented via Feynman diagrams.

![Scattering diagram](https://github.com/mcgen-ct/tutorials/blob/main/.full/mc/figures/scatter.png?raw=1)

Each member of the Feynman diagram represents a mathematical object. In this diagram there are three components: external particle lines in blue, an internal propagator in green, and interaction vertices in red.

$$
\mathcal{M} = \text{i}\left[\color{blue}{\bar{u}_{(3)}} \color{red}{\text{i} \sqrt{4\pi\alpha} \gamma^\mu} \color{blue}{v_{(4)}} \right] \color{green}{\frac{-\text{i} g_{\mu\nu}}{|p_{(0)}|^2}} \left[\color{blue}{\bar{v}_{(2)}} \color{red}{\text{i} \sqrt{4\pi\alpha} \gamma^\nu} \color{blue}{u_{(1)}} \right]
$$

After contracting our indices $\mu$ and $\nu$ we have the following.

$$
\mathcal{M} = \frac{\color{red}{-4\pi\alpha}}{\color{green}{|p_{(0)}|^2}}\left[\color{blue}{\bar{u}_{(3)}} \color{red}{\gamma^\mu} \color{blue}{v_{(4)}} \right] \left[\color{blue}{\bar{v}_{(2)}} \color{red}{\gamma_\mu} \color{blue}{u_{(1)}} \right]
$$

In this diagram, the electron and positron annihilate into an off-shell photon, which then produces a muon and anti-muon final state. There are infinitely more diagrams which can be drawn with the same initial and final state as this diagram, but these diagrams have more factors of $\alpha$, the fine structure constant of $1/137$. Since $\alpha$ is much less than one, these diagrams contribute very little to the summed matrix element, and can oftentimes be neglected. This is what we call a leading order diagram, as these diagrams represent a perturbative expansion about $\alpha$ of the interaction between the initial and final state.

Each line in the diagram, corresponding to a particle, is assigned a momentum. The electron has momentum $p_{(1)}$, the positron $p_{(2)}$, the photon $p_{(0)}$, the muon $p_{(3)}$, and the anti-muon $p_{(4)}$. The photon momentum is determined by conservation of energy and momentum, $p_{(0)} = p_{(1)} + p_{(2)}$. Additionally, each incoming or outgoing particle is assigned a helicity, $\lambda_{(i)}$, which is the spin of the particle projected along the direction of the particle momentum. Fermions are spin $\frac{1}{2}$ and can take on two helicity eigenvalues, either $+1$ or $-1$. The fermion lines, e.g., the electron/positron and muon/anti-muon lines, are Dirac spinors which represent the wave function of the particle.

Our goal is to numerically represent this matrix element, and then integrate a cross-section. Throughout this notebook we will work in the Weyl basis to define our Dirac spinors $u$ and $v$, and the Dirac matrices $\gamma^\mu$.

## Dirac Matrices

The Dirac matrices, $\gamma^\mu$ are defined as,

$$
\gamma^0 = \begin{pmatrix}
  0 & 0 & 1 & 0 \\
  0 & 0 & 0 & 1 \\
  1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
\end{pmatrix}\quad
\gamma^1 = \begin{pmatrix}
  0 & 0 & 0 & 1 \\
  0 & 0 & 1 & 0 \\
  0 & -1 & 0 & 0 \\
  -1 & 0 & 0 & 0 \\
\end{pmatrix}\quad
\gamma^2 = \begin{pmatrix}
  0 & 0 & 0 & -\text{i} \\
  0 & 0 & \text{i} & 0 \\
  0 & \text{i} & 0 & 0 \\
  -\text{i} & 0 & 0 & 0 \\
\end{pmatrix}\quad
\gamma^3 = \begin{pmatrix}
  0 & 0 & 1 & 0 \\
  0 & 0 & 0 & -1 \\
  -1 & 0 & 0 & 0 \\
  0 & 1 & 0 & 0 \\
\end{pmatrix}
$$

and can have their index raised and lowered with the Minkowski metric,

$$
g_{\mu\nu} = \begin{pmatrix}
  1 & 0 & 0 & 0 \\
  0 & -1 & 0 & 0 \\
  0 & 0 & -1 & 0 \\
  0 & 0 & 0 & -1 \\
\end{pmatrix}
$$

just like a four-vector. The repeated index in the matrix element definition for $e^- e^+ \to \mu^- \mu^+$ indicates that a summation should be performed over $\mu$ from $0$ to $3$.

### Exercise: define the Dirac matrices

Using the Weyl basis above, implement the Diract matrices in the skeleton class below. Here, the `FourVector` class transforms under the Minkowski metric using the notation `~v` for `FourVector` `v`.

In [ ]:
###START_EXERCISE
class DiracMatrices(FourVector):
    """
    This class provides the Dirac matrices. Note that this class
    inherits from the 'FourVector' class. This is because the Dirac
    matrices also transform under the Minkowski metric, just like
    standard four-vectors.
    """

    def __init__(self, v0=None, v1=None, v2=None, v3=None):
        """
        Initialize the Dirac matrices.
        """
        # Implement gamma^0.
        g0 = Matrix(
            [0.0, 0.0, 1.0, 0.0],
            [0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0],
        )
        # Implement the remainind Dirac matrices.
        # Initialize the base class with these matrices as the elements of the
        # `FourVector`.
        FourVector.__init__(self, g0, g1, g2, g3)


###STOP_EXERCISE

In [ ]:
###START_SOLUTION
class DiracMatrices(FourVector):
    """
    This class provides the Dirac matrices. Note that this class
    inherits from the 'FourVector' class. This is because the Dirac
    matrices also transform under the Minkowski metric, just like
    standard four-vectors.
    """

    def __init__(self, v0=None, v1=None, v2=None, v3=None):
        """
        Initialize the Dirac matrices. Ideally this would not be mutable.
        """
        g0 = Matrix(
            [0.0, 0.0, 1.0, 0.0],
            [0.0, 0.0, 0.0, 1.0],
            [1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0],
        )
        g1 = Matrix(
            [0.0, 0.0, 0.0, 1.0],
            [0.0, 0.0, 1.0, 0.0],
            [0.0, -1.0, 0.0, 0.0],
            [-1.0, 0.0, 0.0, 0.0],
        )
        g2 = Matrix(
            [0.0, 0.0, 0.0, -1.0j],
            [0.0, 0.0, 1.0j, 0.0],
            [0.0, 1.0j, 0.0, 0.0],
            [-1.0j, 0.0, 0.0, 0.0],
        )
        g3 = Matrix(
            [0.0, 0.0, 1.0, 0.0],
            [0.0, 0.0, 0.0, -1.0],
            [-1.0, 0.0, 0.0, 0.0],
            [0.0, 1.0, 0.0, 0.0],
        )
        FourVector.__init__(self, g0, g1, g2, g3)


###STOP_SOLUTION

### Exercise: use the Dirac matrices

Once a `DiracMatrix` object, say `dm`, the individual matrices can be accessed by the index operator, `[i]`. Print out all four Dirac matrices.

In [ ]:
###START_EXERCISE
###STOP_EXERCISE

In [ ]:
###START_SOLUTION
# Create the Dirac matrices.
dm = DiracMatrices()

# Loop over the Dirac matrices.
for i in range(0, 4):
    print(dm[i])
###STOP_SOLUTION